In [1]:
# !wget https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/malaysia.geojson

In [2]:
# !pip3 install bokeh distributed dask

In [3]:
import json
import geojson
import requests
import re
import os
import numpy as np
import dask.array as da
import dask.dataframe as dd
from shapely.geometry import mapping, shape, MultiPoint
from shapely.ops import cascaded_union

In [4]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:61842 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


In [5]:
with open('data/malaysia.geojson') as fopen:
    malaysia = json.load(fopen)

In [6]:
negeri = {k['properties']['name']:k['geometry'] for k in malaysia['features']}
negeri.keys()

dict_keys(['Federal Territory of Kuala Lumpur', 'Perlis', 'Sabah', 'Federal Territory of Putrajaya', 'Kedah', 'Sarawak', 'Penang', 'Johor', 'Kelantan', 'Melaka', 'Negeri Sembilan', 'Pahang', 'Perak', 'Selangor', 'Terengganu'])

In [7]:
# data from TindakMalaysia is more precise for these states

download = {
    'Kedah': 'https://raw.githubusercontent.com/TindakMalaysia/Kedah-Maps/master/2016/MAP/MIGRATED/result/02-Kedah-New-DM-4326.geojson',
    'Pahang': 'https://raw.githubusercontent.com/TindakMalaysia/Pahang-Maps/master/2016/MAP/MIGRATED/result/07-Pahang-New-DM-4326.geojson',
    'Johor': 'https://raw.githubusercontent.com/TindakMalaysia/Johor-Maps/master/2016/MAP/MIGRATED/result/13-Johor-New-DM-4326.geojson',
    'Perak': 'https://raw.githubusercontent.com/TindakMalaysia/Perak-Maps/master/2016/MAP/MIGRATED/result/06-Perak-New-DM-4326.geojson',
    'Labuan': 'https://raw.githubusercontent.com/TindakMalaysia/Federal-Territories-Maps/master/LABUAN/2016/MAP/MIGRATED/result/14-Labuan-New-DM-4326.geojson',
    'Terengganu': 'https://raw.githubusercontent.com/TindakMalaysia/Terengganu-Maps/master/2016/MAP/MIGRATED/result/04-Terengganu-New-DM-4326.geojson'
}

In [8]:
# for k, v in download.items():
#     os.system(f'wget {v}')

In [9]:
def check(df):
    points = MultiPoint(df.values)
    try:
        ls = list(points.intersection(polygon))
        return [(l.x, l.y) for l in ls]
    except:
        return []

In [10]:
!mkdir data

mkdir: data: File exists


In [11]:
resolutions = {'Sarawak': 0.005, 'Sabah': 0.005}
default_resolution = 0.002

negeris = list(negeri.keys()) + ['Labuan']
for k in negeris:
        
    print(k)
    if k in download:
        print(download[k].split('/')[-1])
        with open(download[k].split('/')[-1]) as fopen:
            data = json.load(fopen)

        state = []
        for d in data['features']:
            d = d['geometry']
            g = geojson.loads(json.dumps(d))
            polygon = shape(g)
            state.append(polygon)
        polygon = cascaded_union(state)
    else:
        s = json.dumps(negeri[k])
        g1 = geojson.loads(s)
        polygon = shape(g1)
    latmin, lonmin, latmax, lonmax = polygon.bounds 
    resolution = resolutions.get(k, default_resolution)
    lat_arange = da.arange(latmin, latmax, resolution)
    lon_arange = da.arange(lonmin, lonmax, resolution)
    x, y = da.meshgrid(lat_arange, lon_arange)
    x = da.round(x, 6)
    y = da.round(y, 6)
    x = da.reshape(x, (-1, 1))
    y = da.reshape(y, (-1, 1))
    concated = da.concatenate([x, y], axis = 1)
    df = dd.from_array(concated)
    df = df.repartition(npartitions = 100)
    check_partitions = df.map_partitions(check, meta=object)
    check_partitions = check_partitions.compute()
    inside = []
    for p in check_partitions:
        inside.extend(p)

    print(k, len(inside))
    with open(f'data/{k}-points.json', 'w') as fopen:
        json.dump(inside, fopen)

Federal Territory of Kuala Lumpur
Federal Territory of Kuala Lumpur 4829
Perlis
Perlis 16628
Sabah
Sabah 240019
Federal Territory of Putrajaya
Federal Territory of Putrajaya 936
Kedah
02-Kedah-New-DM-4326.geojson
Kedah 197233
Sarawak
Sarawak 404284
Penang
Penang 21025
Johor
13-Johor-New-DM-4326.geojson
Johor 387279
Kelantan
Kelantan 306609
Melaka
Melaka 33878
Negeri Sembilan
Negeri Sembilan 135665
Pahang
07-Pahang-New-DM-4326.geojson
Pahang 778418
Perak
06-Perak-New-DM-4326.geojson
Perak 426147
Selangor
Selangor 158857
Terengganu
04-Terengganu-New-DM-4326.geojson
Terengganu 264238
Labuan
14-Labuan-New-DM-4326.geojson
Labuan 2103
